<a href="https://www.kaggle.com/code/yutodennou/competition-lightgbm-and-nn?scriptVersionId=170108612" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

<a id="1"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 1. Import Libs🗃️ </b></div>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from lightgbm import LGBMRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
import tensorflow_probability as tfp

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


<a id="2"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 2. Preprocessing📏 </b></div>

In [2]:
train_dataset = pd.read_csv('/kaggle/input/playground-series-s3e25/train.csv')

In [3]:
features = ['allelectrons_Total', 'density_Total', 'allelectrons_Average',
            'val_e_Average', 'atomicweight_Average', 'ionenergy_Average',
            'el_neg_chi_Average', 'R_vdw_element_Average', 'R_cov_element_Average',
            'zaratio_Average', 'density_Average', 'Hardness']

In [4]:
X = train_dataset[features].drop(columns='Hardness')
y = train_dataset.Hardness

In [5]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
model_pre = LGBMRegressor(max_depth=5,n_estimators=250)
model_pre.fit(X, y)
X_new = X.copy()
X_new['Hardness_pred'] = model_pre.predict(X)

In [7]:
def loss_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=50)

def metric_fn(y_true, y_pred):
    return tfp.stats.percentile(tf.abs(y_true - y_pred), q=100) - tfp.stats.percentile(tf.abs(y_true - y_pred), q=0)

In [8]:
callbacks_list1 = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min',restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00001),
    tf.keras.callbacks.TerminateOnNaN()
] 
callbacks_list2 = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min',restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00002),
    tf.keras.callbacks.TerminateOnNaN()
] 
callbacks_list3 = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=30, verbose=2, mode='min',restore_best_weights=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, min_lr=0.00003),
    tf.keras.callbacks.TerminateOnNaN()
] 

In [9]:
def create_model(hp_learning_rate):
    
    input_layer = tf.keras.Input(shape=(len(features), ))
    x = tf.keras.layers.BatchNormalization(epsilon=0.00001)(input_layer)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    x = tf.keras.layers.Dense(32, activation='relu')(x)
    output_layer = tf.keras.layers.Dense(1)(x)    
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    
    model.compile(optimizer=tf.keras.optimizers.Adam(hp_learning_rate, beta_1=0.5),
                  loss=loss_fn,
                  metrics=metric_fn)
    
    return model

<a id="3"></a>
# <div style="box-shadow: rgba(0, 0, 0, 0.16) 0px 1px 4px inset, rgb(51, 51, 51) 0px 0px 0px 3px inset; padding:20px; font-size:32px; font-family: consolas; text-align:center; display:fill; border-radius:15px;  color:rgb(34, 34, 34);"> <b> 3. Learning🔍 </b></div>

In [10]:
model1 = create_model(0.013)
history1 = model1.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=100,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list1,
                    validation_split=0.1)

Epoch 1/100
293/293 [==============================] - 5s 5ms/step - loss: 1.3017 - metric_fn: 4.7948 - val_loss: 0.5693 - val_metric_fn: 3.3158 - lr: 0.0130
Epoch 2/100
293/293 [==============================] - 1s 4ms/step - loss: 0.8120 - metric_fn: 3.1010 - val_loss: 1.2475 - val_metric_fn: 3.5318 - lr: 0.0130
Epoch 3/100
293/293 [==============================] - 1s 4ms/step - loss: 0.7152 - metric_fn: 2.9044 - val_loss: 0.6347 - val_metric_fn: 3.2084 - lr: 0.0130
Epoch 4/100
293/293 [==============================] - 1s 4ms/step - loss: 0.6678 - metric_fn: 3.0551 - val_loss: 0.6827 - val_metric_fn: 3.3252 - lr: 0.0130
Epoch 5/100
293/293 [==============================] - 1s 4ms/step - loss: 0.5902 - metric_fn: 3.0144 - val_loss: 0.6032 - val_metric_fn: 3.4427 - lr: 0.0104
Epoch 6/100
293/293 [==============================] - 1s 4ms/step - loss: 0.5718 - metric_fn: 3.0809 - val_loss: 0.4699 - val_metric_fn: 3.3473 - lr: 0.0104
Epoch 7/100
293/293 [==============================]

In [11]:
model2 = create_model(0.009)
history2 = model2.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=150,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list2,
                    validation_split=0.1)

Epoch 1/150
293/293 [==============================] - 3s 5ms/step - loss: 1.3677 - metric_fn: 4.5409 - val_loss: 1.0879 - val_metric_fn: 3.8498 - lr: 0.0090
Epoch 2/150
293/293 [==============================] - 1s 4ms/step - loss: 0.8613 - metric_fn: 3.1993 - val_loss: 0.7504 - val_metric_fn: 3.5846 - lr: 0.0090
Epoch 3/150
293/293 [==============================] - 1s 4ms/step - loss: 0.7205 - metric_fn: 3.2442 - val_loss: 0.5131 - val_metric_fn: 3.3701 - lr: 0.0090
Epoch 4/150
293/293 [==============================] - 1s 4ms/step - loss: 0.6231 - metric_fn: 3.0917 - val_loss: 0.5173 - val_metric_fn: 3.4204 - lr: 0.0090
Epoch 5/150
293/293 [==============================] - 1s 4ms/step - loss: 0.5604 - metric_fn: 3.1672 - val_loss: 0.4513 - val_metric_fn: 3.4855 - lr: 0.0090
Epoch 6/150
293/293 [==============================] - 1s 4ms/step - loss: 0.5778 - metric_fn: 3.2760 - val_loss: 0.4442 - val_metric_fn: 3.3244 - lr: 0.0090
Epoch 7/150
293/293 [==============================]

In [12]:
model3 = create_model(0.010)
history3 = model3.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=150,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list3,
                    validation_split=0.1)

Epoch 1/150
293/293 [==============================] - 3s 5ms/step - loss: 1.3539 - metric_fn: 4.4119 - val_loss: 0.7072 - val_metric_fn: 3.3031 - lr: 0.0100
Epoch 2/150
293/293 [==============================] - 1s 4ms/step - loss: 0.8655 - metric_fn: 3.4313 - val_loss: 0.5813 - val_metric_fn: 3.3199 - lr: 0.0100
Epoch 3/150
293/293 [==============================] - 1s 4ms/step - loss: 0.7278 - metric_fn: 3.0412 - val_loss: 0.6139 - val_metric_fn: 3.2831 - lr: 0.0100
Epoch 4/150
293/293 [==============================] - 1s 4ms/step - loss: 0.7079 - metric_fn: 3.0806 - val_loss: 0.7670 - val_metric_fn: 3.4111 - lr: 0.0100
Epoch 5/150
293/293 [==============================] - 1s 4ms/step - loss: 0.6222 - metric_fn: 3.1202 - val_loss: 0.5463 - val_metric_fn: 3.2926 - lr: 0.0100
Epoch 6/150
293/293 [==============================] - 1s 4ms/step - loss: 0.5678 - metric_fn: 3.1338 - val_loss: 0.5215 - val_metric_fn: 3.2970 - lr: 0.0100
Epoch 7/150
293/293 [==============================]

In [13]:
model4 = create_model(0.010)
history4 = model4.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=100,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list1,
                    validation_split=0.1)

Epoch 1/100
293/293 [==============================] - 4s 5ms/step - loss: 1.3255 - metric_fn: 5.7074 - val_loss: 1.6754 - val_metric_fn: 4.5762 - lr: 0.0100
Epoch 2/100
293/293 [==============================] - 1s 4ms/step - loss: 0.9132 - metric_fn: 4.5827 - val_loss: 0.6344 - val_metric_fn: 3.7722 - lr: 0.0100
Epoch 3/100
293/293 [==============================] - 1s 4ms/step - loss: 0.7528 - metric_fn: 3.2491 - val_loss: 0.7417 - val_metric_fn: 4.0189 - lr: 0.0100
Epoch 4/100
293/293 [==============================] - 1s 4ms/step - loss: 0.7569 - metric_fn: 2.9387 - val_loss: 0.5739 - val_metric_fn: 3.2220 - lr: 0.0100
Epoch 5/100
293/293 [==============================] - 1s 4ms/step - loss: 0.6401 - metric_fn: 2.9810 - val_loss: 0.7809 - val_metric_fn: 3.4713 - lr: 0.0100
Epoch 6/100
293/293 [==============================] - 1s 4ms/step - loss: 0.6080 - metric_fn: 3.0863 - val_loss: 0.5812 - val_metric_fn: 3.4444 - lr: 0.0100
Epoch 7/100
293/293 [==============================]

In [14]:
model5 = create_model(0.012)
history5 = model5.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=100,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list2,
                    validation_split=0.1)

Epoch 1/100
293/293 [==============================] - 3s 5ms/step - loss: 1.3208 - metric_fn: 5.2380 - val_loss: 1.3172 - val_metric_fn: 3.8169 - lr: 0.0120
Epoch 2/100
293/293 [==============================] - 1s 4ms/step - loss: 0.8707 - metric_fn: 3.2433 - val_loss: 0.7386 - val_metric_fn: 3.5624 - lr: 0.0120
Epoch 3/100
293/293 [==============================] - 1s 5ms/step - loss: 0.6975 - metric_fn: 2.9455 - val_loss: 0.7205 - val_metric_fn: 3.1345 - lr: 0.0120
Epoch 4/100
293/293 [==============================] - 1s 4ms/step - loss: 0.6784 - metric_fn: 3.2101 - val_loss: 0.6124 - val_metric_fn: 3.4637 - lr: 0.0120
Epoch 5/100
293/293 [==============================] - 1s 4ms/step - loss: 0.6338 - metric_fn: 3.1992 - val_loss: 0.4666 - val_metric_fn: 3.2711 - lr: 0.0120
Epoch 6/100
293/293 [==============================] - 1s 4ms/step - loss: 0.5907 - metric_fn: 3.1910 - val_loss: 1.0490 - val_metric_fn: 3.4651 - lr: 0.0120
Epoch 7/100
293/293 [==============================]

In [15]:
model6 = create_model(0.013)
history6 = model6.fit(X_new.astype('float32'), y.astype('float32'),
                    epochs=100,
                    class_weight=model_pre.class_weight,
                    callbacks=callbacks_list3,
                    validation_split=0.1)

Epoch 1/100
293/293 [==============================] - 3s 5ms/step - loss: 1.3077 - metric_fn: 4.5185 - val_loss: 0.6443 - val_metric_fn: 3.0674 - lr: 0.0130
Epoch 2/100
293/293 [==============================] - 1s 4ms/step - loss: 0.9125 - metric_fn: 3.9364 - val_loss: 0.8661 - val_metric_fn: 3.7470 - lr: 0.0130
Epoch 3/100
293/293 [==============================] - 1s 4ms/step - loss: 0.7704 - metric_fn: 4.2716 - val_loss: 0.5802 - val_metric_fn: 3.3912 - lr: 0.0130
Epoch 4/100
293/293 [==============================] - 1s 4ms/step - loss: 0.6224 - metric_fn: 3.3713 - val_loss: 0.6479 - val_metric_fn: 3.3753 - lr: 0.0130
Epoch 5/100
293/293 [==============================] - 1s 4ms/step - loss: 0.5543 - metric_fn: 3.2607 - val_loss: 0.5345 - val_metric_fn: 3.4703 - lr: 0.0130
Epoch 6/100
293/293 [==============================] - 1s 4ms/step - loss: 0.5818 - metric_fn: 3.3482 - val_loss: 0.4681 - val_metric_fn: 3.6690 - lr: 0.0130
Epoch 7/100
293/293 [==============================]

In [16]:
sample_submission = pd.read_csv('/kaggle/input/playground-series-s3e25/sample_submission.csv')
test_dataset = pd.read_csv('/kaggle/input/playground-series-s3e25/test.csv')

In [17]:

test_dataset['Hardness_pred'] = model_pre.predict(test_dataset.astype('float32').drop(columns='id'))
test_dataset["Hardness"] = model1.predict(test_dataset.astype('float32').drop(columns='id'))*0.3+model2.predict(test_dataset.astype('float32').drop(columns='id'))*0.2+model3.predict(test_dataset.astype('float32').drop(columns='id'))*0.1+model4.predict(test_dataset.astype('float32').drop(columns='id'))*0.2+model5.predict(test_dataset.astype('float32').drop(columns='id'))*0.1+model6.predict(test_dataset.astype('float32').drop(columns='id'))*0.1
sample_submission = test_dataset[['id', "Hardness"]]
sample_submission.to_csv("submission.csv", index=False)

217/217 [==============================] - 0s 1ms/step


In [18]:
sample_submission

,id,Hardness
0,10407,2.366532
1,10408,2.326413
2,10409,5.746166
3,10410,3.569181
4,10411,5.746023
...,...,...
6934,17341,5.745876
6935,17342,3.521794
6936,17343,5.733092
6937,17344,5.502481
